In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Что тут есть:  
- чтение из .txt
- удаление emoji
- лемматизация слов
- самые часто встречаемые и их удаление


Метрика качества:

In [2]:
def calculate_metrics(self, submit):
    df = self.answers.join(submit, on='id')

    y_true = []
    y_scores = []
    for idx, row in df.iterrows():
        answer_labels = row['labels'].split(',')
        y_true.append([1 if label in answer_labels else 0 for label in LABELS])
        y_scores.append([row['normal'], row['insult'], row['obscenity'], row['threat']])

## Чтение данных из файла txt

In [2]:
df = pd.DataFrame(columns = [['id', 'text', 'normal','insult','obscenity','threat']]);
j=0;

f = open("data/train.txt", "r", encoding='utf-8')
for line in f:
    nor, ins, obs, thr = [0,0,0,0]
    l = line.split('\t')
    em_List = (l[1:-1])
    for i in em_List:
        if i=='__label__NORMAL':
            nor+=1
        elif i=='__label__INSULT':
            ins+=1
        elif i=='__label__OBSCENITY':
            obs+=1
        else:
            thr+=1
    df.loc[j]=[int(l[0]),l[-1][:-1],nor,ins,obs,thr]
    j += 1
f.close()

KeyboardInterrupt: 

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
train_file_lenght=148775

In [ ]:
df.head(2000).to_csv('short_train.csv')

Файл очень большой, поэтомц работать пока буду только с первыми 2000 элементами, которые вынесены в отдельный файд short_train.csv

In [83]:
df = pd.read_csv('short_train.csv', usecols=['id', 'text', 'normal','insult','obscenity','threat'])
df.tail()

,id,text,normal,insult,obscenity,threat
1995,15834,"людей искиства тянет ,он не спать не есть небу...",1,0,0,0
1996,64370,какой мудак старое кино переименновывает посто...,0,1,0,0
1997,17080,спит он под кондиционером! я в восторге!,1,0,0,0
1998,237178,могу пнуть по яйцам и отлетит дебил твоя голова,0,1,0,1
1999,26969,юра ефремов подонок и негодяй к тому же подлей...,0,1,0,0


# emoji

In [42]:
import emoji

In [54]:
df.iloc[94]

id                           69952
text         это обоюдное решение✌
normal                           1
insult                           0
obscenity                        0
threat                           0
Name: 94, dtype: object

In [55]:
encode = df.iloc[94]['text']
encode

'это обоюдное решение✌'

In [111]:
def delEmoji(text):
    allchars = [str for str in text]
    lst = [c for c in allchars if c not in emoji.UNICODE_EMOJI]
    lst = ''.join(lst)
    if lst !='':
        return lst
    else:
        return None

In [86]:
df.iloc[1130]

id                         240196
text         офигенная бобомчка 😲
normal                          1
insult                          0
obscenity                       0
threat                          0
Name: 1130, dtype: object

In [112]:
a = delEmoji(df.iloc[1130]['text'])
a

'офигенная бобомчка '

# Обработка текста

S.lower

In [8]:
import re
from pymorphy2 import MorphAnalyzer

In [6]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

In [9]:
morph = MorphAnalyzer()

In [118]:
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        token = token.strip()
        token = delEmoji(token)
        if token:
            token = morph.normal_forms(token)[0]
        else:
            continue
            
        tokens.append(token)
    if len(tokens) >= 1:
        return tokens
    return None

In [119]:
a = lemmatize(df.iloc[1130]['text'])
a

['офигенный', 'бобомчка']

In [120]:
data = df['text'].apply(lemmatize)

In [121]:
data.head()

0                    [дворник, надо, тоже, уничтожить]
1    [мой, старший, неделя, шипеть, не, принимать, ...
2                        [полностью, с, вы, согласный]
3          [хоть, нога, вверх, ничего, не, измениться]
4                     [а, что, значит, левый, ребёнок]
Name: text, dtype: object

In [122]:
data = data.dropna()

# Самые распространённые слова в датасете

In [27]:
from collections import defaultdict
word_freq = defaultdict(int)
for tokens in data[:]:
    for token in tokens:
        word_freq[token] += 1

In [28]:
len(word_freq)

6293

In [66]:
topWords = sorted(word_freq, key=word_freq.get, reverse=True)[:30]
topWords

['и',
 'в',
 'не',
 'а',
 'что',
 'на',
 'я',
 'с',
 'быть',
 'это',
 'он',
 'вы',
 'как',
 'всё',
 'ты',
 'у',
 'они',
 'то',
 'такой',
 'так',
 'за',
 'мы',
 'по',
 'весь',
 'этот',
 'человек',
 'о',
 'кто',
 'какой',
 'бы']

эти слова не являются сильно информативными, их можно удалить

In [71]:
def delTopWords(tokens):
    return [token for token in tokens if token not in topWords]

In [123]:
data = data.apply(delTopWords)

In [124]:
data.head()

0                    [дворник, надо, тоже, уничтожить]
1    [мой, старший, неделя, шипеть, принимать, подк...
2                               [полностью, согласный]
3              [хоть, нога, вверх, ничего, измениться]
4                             [значит, левый, ребёнок]
Name: text, dtype: object

In [125]:
df['text'] = data
df.head()

,id,text,normal,insult,obscenity,threat
0,41127,"[дворник, надо, тоже, уничтожить]",0,0,0,1
1,6812,"[мой, старший, неделя, шипеть, принимать, подк...",1,0,0,0
2,6256,"[полностью, согласный]",1,0,0,0
3,189636,"[хоть, нога, вверх, ничего, измениться]",1,0,0,0
4,99053,"[значит, левый, ребёнок]",1,0,0,0


In [126]:
df.tail()

,id,text,normal,insult,obscenity,threat
1995,15834,"[искиство, тянуть, спать, есть, небыть, онбыть...",1,0,0,0
1996,64370,"[мудак, старое, кино, переименновывать, постоя...",0,1,0,0
1997,17080,"[спать, под, кондиционер, восторг]",1,0,0,0
1998,237178,"[мочь, пнуть, яйцо, отлететь, дебил, твой, гол...",0,1,0,1
1999,26969,"[юра, ефрем, подонок, негодяй, к, тот, же, под...",0,1,0,0


## Разделение на обучающую и тестовую выборки

In [76]:
from sklearn.model_selection import train_test_split

In [77]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], 
                                                    df.drop('text', axis=1), 
                                                    test_size=0.33,
                                                    random_state=42)

In [78]:
y_train

,id,normal,insult,obscenity,threat
81,180341,0,1,0,0
915,241550,0,1,0,0
1018,45802,0,1,0,0
380,178744,0,0,0,1
1029,76653,1,0,0,0
...,...,...,...,...,...
1130,240196,1,0,0,0
1294,80655,1,0,0,0
860,82203,1,0,0,0
1459,193207,1,0,0,0


In [79]:
X_train

81             [рот, свой, закрыть, подстилка, путинский]
915     [рада, есть, ранний, до, пандемия, вынудить, у...
1018    [заести, сука, когда, же, нажраться, падла, бл...
380                                    [сдохнуть, скорее]
1029    [здравствуйте, срочно, кизил, нужный, но, немн...
                              ...                        
1130                             [офигенный, бобомчка, 😲]
1294    [кошмар, срань, интернетовскай, посмотреть, св...
860     [делать, если, заяц, прописаться, дача, выгрыз...
1459                          [серпатый, торцовый, класс]
1126    [познакомиться, парень, сказать, пока, платье,...
Name: text, Length: 1340, dtype: object

## Преобразование текстовых данных   
#### CountVectorizer

In [139]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer()

In [149]:
comments_vec_train = count_vec.fit_transform(X_train)

In [150]:
comments_vec_train

<1340x6575 sparse matrix of type '<class 'numpy.int64'>'
	with 13895 stored elements in Compressed Sparse Row format>

In [151]:
comments_vec_test = count_vec.transform(X_test)

In [152]:
from sklearn.linear_model import LogisticRegression

In [155]:
model1 = LogisticRegression()
model1.fit(comments_vec_train,y_train['normal'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [160]:
model2 = LogisticRegression()
model2.fit(comments_vec_train,y_train['insult'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [161]:
model3 = LogisticRegression()
model3.fit(comments_vec_train,y_train['obscenity'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [162]:
model4 = LogisticRegression()
model4.fit(comments_vec_train,y_train['threat'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [163]:
p1 = model1.predict(comments_vec_test)
p2 = model2.predict(comments_vec_test)
p3 = model3.predict(comments_vec_test)
p4 = model4.predict(comments_vec_test)

In [166]:
from sklearn.metrics import f1_score

In [170]:
f1_score(y_test['normal'], p1)

0.921875

In [171]:
F1 = np.array([f1_score(y_test['normal'], p1),
         f1_score(y_test['insult'], p2),
         f1_score(y_test['obscenity'], p3),
         f1_score(y_test['threat'], p4),

])
F1.mean() # F1

0.341399485930736